In [21]:
import sys
import os
sys.path.append(os.path.abspath('..'))

from data_managment import datasets, nuscenes
from data_managment.data_modules import collate_ultralidar
from model_managment.helpers import load_model
from model_managment.models import UltraLidarModel
from data_managment.data_modules import LidarTokenizerModule

import matplotlib.image as mpimg
from PIL import Image

import matplotlib
import numpy as np
import plotly.graph_objects as go
import torch
from torch.utils.data import DataLoader
import pytorch_lightning as pl


In [22]:
configs = ["../model_managment/config_FINAL_ul_base.yaml",
               "../model_managment/config_FINAL_ul_fsq.yaml",
               "../model_managment/config_FINAL_ul_base_1k.yaml",
               "../model_managment/config_FINAL_ul_base_1k_4k.yaml"]

models = ["../slurm_logs/ul_FINAL_vq_241116_2328/models/ultralidar-epoch=11-step=39999.ckpt",
            "../slurm_logs/ul_FINAL_fsq_241116_2325/models/ultralidar-epoch=11-step=39999.ckpt",
            "../slurm_logs/ul_FINAL_vq_1k_241117_1647/models/ultralidar-epoch=11-step=39999.ckpt",
            "../slurm_logs/ul_FINAL_vq_1k_4k_241117_2326/models/ultralidar-epoch=11-step=39999.ckpt"]
    
assert len(configs) == len(models)

In [23]:
k = 0 # number of model from above lists
model = load_model(configs[k])
checkpoint = torch.load(models[k], map_location=torch.device('cpu'))
model.load_state_dict(checkpoint["state_dict"])
model.eval()

data = LidarTokenizerModule(dataroot = '/mnt/data/Public_datasets/nuScenes/',
                      batch_size = 1,
                      num_workers = 0,
                      info_file_path = "../data_managment/nuscenes_info_plot.json",    # specifies which files are train/val/test
                      mode="ultralidar",
                      dataset="nuscenes",
                      config_path=configs[k][3:],
                      version="v1.0-test"
                      )   
  
trainer = pl.Trainer(gpus=0)
predictions = trainer.predict(model, datamodule=data)

/tmp/ipykernel_902763/2677611782.py:3: FutureWarning:

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available

Predicting: 0it [00:00, ?it/s]

In [24]:
import plotly.io as pio
pio.renderers.default = 'browser'

i = 0

metrics = predictions[i][0]
pcd_in = predictions[i][1][0][:,:3]
pcd_out = predictions[i][2]
voxels_pcd_in = predictions[i][3]

print(f"IOU             : {round(float(metrics['test/lidar_rec_iou'].item()),3)}")
print(f"Code utilization: {round(float(metrics['test/code_util'].item()),3)}")
print(f"Code uniformity : {round(float(metrics['test/code_uniformity'].item()),3)}")

fig = go.Figure(data=[go.Scatter3d(
    x=pcd_in[:,0],
    y=pcd_in[:,1],
    z=pcd_in[:,2],
    mode='markers',
    marker=dict(size=2, color='blue')
)])

fig.add_trace(go.Scatter3d(
    x=pcd_out[:,0],
    y=pcd_out[:,1],
    z=pcd_out[:,2],
    mode='markers',
    marker=dict(size=2, color='green')
))

fig.add_trace(go.Scatter3d(
    x=voxels_pcd_in[:,0],
    y=voxels_pcd_in[:,1],
    z=voxels_pcd_in[:,2],
    mode='markers',
    marker=dict(size=2, color='red')
))

""" cube_x = [0, 100, 100, 0, 0,        0, 100, 100, 0, 0, 100,100    ,100,100,0,0]
cube_y = [-50, -50, 50, 50, -50,    -50, -50, 50, 50, -50, -50,-50, 50,50,50,50]
cube_z = [-2, -2, -2, -2, -2,       18, 18, 18, 18, 18, 18,    -2, -2, 18, 18,-2]

# Adding the cube outline
fig.add_trace(go.Scatter3d(
    x=cube_x,
    y=cube_y,
    z=cube_z,
    mode='lines',
    line=dict(color='red', width=2),
    name='Occupancy Grid Outline'
)) """

fig.update_layout(
    scene=dict(
        xaxis=dict(nticks=10, range=[-70, 70], backgroundcolor="white"),
        yaxis=dict(nticks=10, range=[-70, 70], backgroundcolor="white"),
        zaxis=dict(nticks=10, range=[-70, 70], backgroundcolor="white"),
        aspectmode='cube'
    ),
    paper_bgcolor="white",  # Background color outside the 3D scene
    plot_bgcolor="white"    # Background color inside the 3D scene
)

fig.show()

IOU             : 0.477
Code utilization: 0.983
Code uniformity : 0.139
